<a href="https://colab.research.google.com/github/juliatessler/1s2023-unicamp-dl-for-search-systems/blob/main/10-trade-offs/10_tradeoffs_doc2query%2Bbaseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 9- Multi-stage system comparision
by Júlia Tessler

In [ ]:
!pip install transformers datasets -q
!pip install sentence-transformers -q
!pip install pyserini -q
!pip install faiss-gpu -q
!pip install jsonlines -q
!pip install evaluate -q
!pip install trectools -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 104.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 119.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 81.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 295.0 kB/s eta 0:00:00
  Preparing metadata (setup.py) 

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)
workdir = '/content/drive/MyDrive/Unicamp/DL_applied_to_IR/Notebooks'

Mounted at /content/drive


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue May  9 21:34:37 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
import numpy as np
import pandas as pd
import json
import jsonlines
import time

from transformers import (
    T5Tokenizer, 
    T5ForConditionalGeneration
)
from pyserini.search.lucene import LuceneSearcher
from evaluate import load
from collections import defaultdict
from tqdm import tqdm

## Dataset retrieval


### Test dataset

In [ ]:
!wget https://huggingface.co/datasets/BeIR/trec-covid-qrels/raw/main/test.tsv

--2023-05-09 21:35:12--  https://huggingface.co/datasets/BeIR/trec-covid-qrels/raw/main/test.tsv
Resolving huggingface.co (huggingface.co)... 18.155.68.38, 18.155.68.121, 18.155.68.116, ...
Connecting to huggingface.co (huggingface.co)|18.155.68.38|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 980831 (958K) [text/plain]
Saving to: ‘test.tsv’

test.tsv            100%[===================>] 957.84K  1.10MB/s    in 0.9s    

2023-05-09 21:35:14 (1.10 MB/s) - ‘test.tsv’ saved [980831/980831]



In [ ]:
!mv test.tsv {workdir}/trec-covid

In [ ]:
qrel = pd.read_csv(f"{workdir}/trec-covid/test.tsv", 
                   sep = "\t",
                   header=None, 
                   skiprows = 1, 
                   names = ["query", "docid", "rel"])
qrel["q0"] = "q0"
qrel = qrel.to_dict(orient = "list")

### Queries

In [ ]:
!wget https://huggingface.co/datasets/BeIR/trec-covid/resolve/main/queries.jsonl.gz

--2023-05-09 21:35:15--  https://huggingface.co/datasets/BeIR/trec-covid/resolve/main/queries.jsonl.gz
Resolving huggingface.co (huggingface.co)... 18.155.68.116, 18.155.68.121, 18.155.68.38, ...
Connecting to huggingface.co (huggingface.co)|18.155.68.116|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/a8/10/a810e88b0e7b233be82b89c1fa6ec2d75efc6d55784c2ada9dcac8434a634f3a/9eadcc2cdf140addc9dae83648bb2c6611f5e4b66eaed7475fa5a0ca48eda371?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27queries.jsonl.gz%3B+filename%3D%22queries.jsonl.gz%22%3B&response-content-type=application%2Fgzip&Expires=1683921275&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2E4LzEwL2E4MTBlODhiMGU3YjIzM2JlODJiODljMWZhNmVjMmQ3NWVmYzZkNTU3ODRjMmFkYTlkY2FjODQzNGE2MzRmM2EvOWVhZGNjMmNkZjE0MGFkZGM5ZGFlODM2NDhiYjJjNjYxMWY1ZTRiNjZlYWVkNzQ3NWZhNWEwY2E0OGVkYTM3MT9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW

In [ ]:
!mv queries.jsonl.gz {workdir}/trec-covid
!gunzip {workdir}/trec-covid/queries.jsonl.gz

gzip: /content/drive/MyDrive/Unicamp/DL_applied_to_IR/Notebooks/trec-covid/queries.jsonl already exists; do you wish to overwrite (y or n)? y


In [ ]:
query_ids = []
query_texts = []

with jsonlines.open(f"{workdir}/trec-covid/queries.jsonl") as reader:
  for item in reader:
    id = item["_id"]
    query_ids.append(id)
    text = item["text"]
    query_texts.append(text)

### Corpus

In [ ]:
!wget https://huggingface.co/datasets/BeIR/trec-covid/resolve/main/corpus.jsonl.gz

--2023-05-09 21:35:34--  https://huggingface.co/datasets/BeIR/trec-covid/resolve/main/corpus.jsonl.gz
Resolving huggingface.co (huggingface.co)... 18.155.68.38, 18.155.68.44, 18.155.68.116, ...
Connecting to huggingface.co (huggingface.co)|18.155.68.38|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/a8/10/a810e88b0e7b233be82b89c1fa6ec2d75efc6d55784c2ada9dcac8434a634f3a/e9e97686e3138eaff989f67c04cd32e8f8f4c0d4857187e3f180275b23e24e85?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27corpus.jsonl.gz%3B+filename%3D%22corpus.jsonl.gz%22%3B&response-content-type=application%2Fgzip&Expires=1683927335&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2E4LzEwL2E4MTBlODhiMGU3YjIzM2JlODJiODljMWZhNmVjMmQ3NWVmYzZkNTU3ODRjMmFkYTlkY2FjODQzNGE2MzRmM2EvZTllOTc2ODZlMzEzOGVhZmY5ODlmNjdjMDRjZDMyZThmOGY0YzBkNDg1NzE4N2UzZjE4MDI3NWIyM2UyNGU4NT9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSo

In [ ]:
!mv corpus.jsonl.gz {workdir}/trec-covid
!gunzip {workdir}/trec-covid/corpus.jsonl.gz

gzip: /content/drive/MyDrive/Unicamp/DL_applied_to_IR/Notebooks/trec-covid/corpus.jsonl already exists; do you wish to overwrite (y or n)? y


In [ ]:
passage_ids = []
passage_texts = []
id_to_passage = dict()
id_to_index = dict()

with jsonlines.open(f"{workdir}/trec-covid/corpus.jsonl") as reader:
  for item in reader:
    id = item["_id"]
    id_to_index[id] = len(passage_ids)
    passage_ids.append(id)
    text = item["title"] + ' ' + item["text"]
    passage_texts.append(text)
    id_to_passage[id] = text

## BM25 baseline

In [ ]:
def search_with_bm25(query, k = 1000, index_name = 'beir-v1.0.0-trec-covid.flat'):
  if index_name == 'beir-v1.0.0-trec-covid.flat':   
    searcher = LuceneSearcher.from_prebuilt_index(index_name)
  else:
    searcher = LuceneSearcher(index_name)
  hits = searcher.search(query, k)
  return hits

In [ ]:
def eval_ndcg10(run):
  trec_eval = load("trec_eval")
  results = trec_eval.compute(predictions=[run], references=[qrel])
  return results['NDCG@10'] 

In [ ]:
def run_bm25(test_ids, test_corpus, top_k = 1000):
  run = defaultdict(list)

  for id, query in tqdm(zip(test_ids, test_corpus)):
    bm25_hits = search_with_bm25(query, k = top_k)
    ids = [json.loads(bm25_hits[i].raw)['_id'] for i in range(len(bm25_hits))]
    run["query"] += [id] * top_k
    run["docid"] += ids
    run["score"] += [1] * top_k
    run["q0"] += ["q0"] * top_k
    run["rank"] += list(range(1,top_k+1))
    run["system"] += ['bm25'] * top_k

  return run

In [ ]:
run_bm25 = run_bm25(query_ids, query_texts)

0it [00:00, ?it/s]


lucene-index.beir-v1.0.0-trec-covid.flat.20221116.505594.tar.gz: 0.00B [00:00, ?B/s]
lucene-index.beir-v1.0.0-trec-covid.flat.20221116.505594.tar.gz:   0%|          | 8.00k/216M [00:01<11:26:51, 5.49kB/s]
lucene-index.beir-v1.0.0-trec-covid.flat.20221116.505594.tar.gz:   0%|          | 40.0k/216M [00:01<1:59:59, 31.4kB/s] 
lucene-index.beir-v1.0.0-trec-covid.flat.20221116.505594.tar.gz:   0%|          | 88.0k/216M [00:01<54:34, 69.1kB/s]  
lucene-index.beir-v1.0.0-trec-covid.flat.20221116.505594.tar.gz:   0%|          | 200k/216M [00:02<22:46, 165kB/s]  
lucene-index.beir-v1.0.0-trec-covid.flat.20221116.505594.tar.gz:   0%|          | 424k/216M [00:02<10:21, 363kB/s]
lucene-index.beir-v1.0.0-trec-covid.flat.20221116.505594.tar.gz:   0%|          | 856k/216M [00:02<05:00, 751kB/s]
lucene-index.beir-v1.0.0-trec-covid.flat.20221116.505594.tar.gz:   1%|          | 1.71M/216M [00:02<02:21, 1.58MB/s]
lucene-index.beir-v1.0.0-trec-covid.flat.20221116.505594.tar.gz:   2%|▏         | 3.43M/216

In [ ]:
baseline_bm25_ndcg = eval_ndcg10(run_bm25)
baseline_bm25_ndcg

0.5946917010118077

## docTTTTTquery
Using much from [here](https://github.com/castorini/docTTTTTquery).

In [ ]:
tokenizer = T5Tokenizer.from_pretrained('castorini/doc2query-t5-base-msmarco')
model = T5ForConditionalGeneration.from_pretrained('castorini/doc2query-t5-base-msmarco')
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

### Document expansion

In [ ]:
batch_size = 32
NUM_SAMPLE_QUERIES = 3
doc2queries = defaultdict(list)
    
last_saved_batch = -1
extended_docs = []
gen_greedy = True
gen_beam = True

In [ ]:
def predict(texts, do_sample = False, num_beams = 1, num_return_sequences = 1):
  input_ids = tokenizer(texts, 
                        return_tensors = "pt", 
                        padding = "max_length", 
                        truncation = True).input_ids.to(device)
  sequence_ids = model.generate(input_ids, 
                                do_sample = do_sample,
                                num_beams = num_beams, 
                                max_new_tokens=128, 
                                num_return_sequences = num_return_sequences)
  sequences = tokenizer.batch_decode(sequence_ids, skip_special_tokens=True)
  return sequences

In [ ]:
def add_queries(ids, queries, num_return_sequences=1):
  for i, q in enumerate(queries):
    doc2queries[ids[i]].append(q)

In [ ]:
# Kudos Monique!
for i in tqdm(range(0, len(passage_texts), batch_size)):
  #texts = [doc["text"] for doc in unprocessed_corpus[i:i+batch_size]]
  #Last test: expand titles too
  texts = passage_texts[i:i+batch_size]
  ids = passage_ids[i:i+batch_size]

  #Generates a batch of 1 greedy query per document
  if gen_greedy == True:
    greedy_queries = predict(texts)
    add_queries(ids, greedy_queries)
  
  if gen_beam == True:
    #Generates a batch of 1 query per document by using beam search
    beam_search_queries = predict(texts, num_beams=2)
    add_queries(ids, beam_search_queries)
  
  #Generates 3 queries per document by using random sampling
  sampled_queries = predict(texts, do_sample=True, 
                            num_return_sequences=NUM_SAMPLE_QUERIES)
  ids_list = []
  for id in ids:
    ids_list += [id]*NUM_SAMPLE_QUERIES

  add_queries(ids_list, sampled_queries, num_return_sequences=NUM_SAMPLE_QUERIES)    

100%|██████████| 5355/5355 [14:12:39<00:00,  9.55s/it]


In [ ]:
doc2queries

defaultdict(list,
            {'ug7v899j': ['what is the most common comorbidity of mycoplasma pneumoniae',
              'what is the most common comorbidity of mycoplasma pneumoniae',
              'what is the most common comorbidity of mycoplasma pneumoniae',
              'what is the most common comorbidity of mycoplasma pneumoniae'],
             '02tnwd4m': ['what is nitric oxide',
              'is nitric oxide an inflammatory mediator',
              'what is nitric oxide',
              'is nitric oxide an inflammatory mediator'],
             'ejv2xln0': ['what is surfactant protein d',
              'what is surfactant protein d',
              'what is surfactant protein d',
              'what is surfactant protein d'],
             '2b73a28n': ['what is endothelin',
              'what is endothelin',
              'what is endothelin',
              'what is endothelin'],
             '9785vg6d': ['what is the response of respiratory epithelial cells to pneumovirus',
 

In [ ]:
import pickle

with open(f'{workdir}/doc2queries.pickle', 'wb') as f:
  pickle.dump(doc2queries, f)

In [ ]:
corpus = []

with open(f"{workdir}/trec-covid/corpus.jsonl", 'r', encoding='utf-8') as f:
    # Read the file line by line
    lines = f.readlines()
    
    # Parse each line as a JSON object and store it in a list
    for line in lines:
        corpus.append(json.loads(line))

doc_id_to_text_and_title = {doc["_id"]: (doc["text"], doc["title"]) for doc in corpus}

In [ ]:
extended_docs = []

for doc_id, queries in doc2queries.items():
  text, title = doc_id_to_text_and_title[doc_id]
  text += '\n' + '\n'.join(queries)
  extended_docs.append({"id":doc_id, "contents": title + '\n' + text})
     

In [ ]:
json_batch_size = len(corpus)//10

In [ ]:
j=0

for i in tqdm(range(0, len(extended_docs), json_batch_size)):
  filename = f"{workdir}/trec-covid/collections/json_{j}.json"
  print(filename)
  with jsonlines.open(filename, mode='w') as writer:
    #for item in all_docs[i:i+json_batch_size]:
    for item in extended_docs[i:i+json_batch_size]:
      writer.write(item)
  j+=1

  0%|          | 0/11 [00:00<?, ?it/s]

/content/drive/MyDrive/Unicamp/DL_applied_to_IR/Notebooks/trec-covid/collections/json_0.json


  9%|▉         | 1/11 [00:00<00:03,  3.12it/s]

/content/drive/MyDrive/Unicamp/DL_applied_to_IR/Notebooks/trec-covid/collections/json_1.json


 18%|█▊        | 2/11 [00:00<00:02,  3.32it/s]

...

/content/drive/MyDrive/Unicamp/DL_applied_to_IR/Notebooks/trec-covid/collections/json_8.json


 82%|████████▏ | 9/11 [00:02<00:00,  3.23it/s]

/content/drive/MyDrive/Unicamp/DL_applied_to_IR/Notebooks/trec-covid/collections/json_9.json


100%|██████████| 11/11 [00:03<00:00,  3.47it/s]

/content/drive/MyDrive/Unicamp/DL_applied_to_IR/Notebooks/trec-covid/collections/json_10.json


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input {workdir}/trec-covid/collections \
  --index {workdir}/indexes/doc2query_expasion \
  --generator DefaultLuceneDocumentGenerator \
  --threads 9 \
  --storePositions --storeDocvectors --storeRaw

2023-05-10 14:13:27,658 INFO  [main] index.IndexCollection (IndexCollection.java:380) - Setting log level to INFO
2023-05-10 14:13:27,660 INFO  [main] index.IndexCollection (IndexCollection.java:383) - Starting indexer...
2023-05-10 14:13:27,660 INFO  [main] index.IndexCollection (IndexCollection.java:384) - ============ Loading Parameters ============
2023-05-10 14:13:27,661 INFO  [main] index.IndexCollection (IndexCollection.java:385) - DocumentCollection path: /content/drive/MyDrive/Unicamp/DL_applied_to_IR/Notebooks/trec-covid/collections
2023-05-10 14:13:27,661 INFO  [main] index.IndexCollection (IndexCollection.java:386) - CollectionClass: JsonCollection
2023-05-10 14:13:27,661 INFO  [main] index.IndexCollection (IndexCollection.java:387) - Generator: DefaultLuceneDocumentGenerator
2023-05-10 14:13:27,662 INFO  [main] index.IndexCollection (IndexCollection.java:388) - Threads: 9
2023-05-10 14:13:27,662 INFO  [main] index.IndexCollection (IndexCollection.java:389) - Language: en
2

In [ ]:
with open(f'{workdir}/trec-covid/queries.tsv','w') as f_out:
  with open(f'{workdir}/trec-covid/queries.jsonl', 'r') as f:
    for line in f:
      data = json.loads(line)
      id = data['_id']
      text = data['text']
      f_out.write(f'{id}\t{text}\n')

In [ ]:
!python -m pyserini.search.lucene \
  --index {workdir}/indexes/doc2query_expasion \
  --topics {workdir}/trec-covid/queries.tsv \
  --output {workdir}/runs/run.trec-covid.bm25tuned-doc2query_expansion.txt \
  --hits 1000 \
  --bm25

Running /content/drive/MyDrive/Unicamp/DL_applied_to_IR/Notebooks/trec-covid/queries.tsv topics, saving to /content/drive/MyDrive/Unicamp/DL_applied_to_IR/Notebooks/runs/run.trec-covid.bm25tuned-doc2query_expansion.txt...
100% 50/50 [00:04<00:00, 11.09it/s]


In [ ]:
run = pd.read_csv(f"{workdir}/runs/run.trec-covid.bm25tuned-doc2query_expansion.txt", 
                  sep="\s+", 
                  names=["query", "q0", "docid", "rank", "score", "system"])
run = run.to_dict(orient="list")

In [ ]:
doc2queryndcg = eval_ndcg10(run)
doc2queryndcg

0.6687910737353352

In [ ]:
searcher = LuceneSearcher(f'{workdir}/indexes/doc2query_expasion')

In [ ]:
%%time
hits = searcher.search('covid serious illess')

CPU times: user 20.9 ms, sys: 3 ms, total: 23.9 ms
Wall time: 8.14 ms


In [ ]:
# Print the first 10 hits:
for i in range(0, 10):
    print(f'{i+1:2} {hits[i].docid:15} {hits[i].score:.5f}')

 1 f44vzv4x        4.40890
 2 5rxmw7ws        4.36340
 3 k20ym4u1        4.33040
 4 q1akwoo1        4.20880
 5 30axk468        4.19300
 6 4bne9qup        4.18580
 7 knv4ljyr        4.17070
 8 vc3us86f        4.16200
 9 1ui0p8eu        4.15830
10 7ckw2969        4.14840
